# Graph Models and Hypothesis Testing

In this practical we will discover how use some simple models to generate synthetic networks and how we can use these models to test hypotheses about observed networks.

Some of the libraries are difficult to install on some systems, so it is best to open this notebook in Colab [![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jgarciab/NetworkScience/blob/main/Practicals/day2a_graph_models/Graph_models_and_hypothesis_testing_2025.ipynb)

**You cannot save changes in this notebook, you need to save a copy to your Google Drive: `File` -> `Save a copy in Drive`.**

First, we must install a useful library for analysing networks called [graph-tool](https://graph-tool.skewed.de/)...

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
!mamba install -q graph-tool

Next we can import graph-tool and some other useful libraries.

In [ ]:
import graph_tool.all as gt
from graph_tool import topology, inference, generation, stats, correlations
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from scipy.stats import binomtest

### PART I: Regular and random networks

In the first part of this practical, we will consider some simple network models and the properties of the networks they generate.

The graph-tool documentation for these network models is [here](https://graph-tool.skewed.de/static/doc/generation.html). **It will be useful to refer to the documentation to understand and use the code below**

#### Regular graphs and lattices
Regular graphs are networks in which all nodes have the same degree ($k$).
A lattice graph is a network in which all the nodes are arranged in a regular pattern, such as a ring or a square. Lattice graphs are often regular as all nodes have the same degree, with the exception of boundary nodes.

The following code generates two examples of these types of graph: a square lattice ($k=4$, except boundary nodes) and a regular ring lattice ($k=2$)

In [ ]:
# generate square lattice
g = gt.lattice([10,10])
# create layout
pos = gt.sfdp_layout(g, cooling_step=0.95, epsilon=1e-2)
# draw the network
gt.graph_draw(g, pos=pos)

# generate ring lattice
g = gt.circular_graph(20, 2)
# create layout
pos = gt.sfdp_layout(g, cooling_step=0.95)
# draw the network
gt.graph_draw(g, pos=pos)

We know that regular graphs have constant degree. We can check the level of clustering (closed triangles) and average path length (diameter) using the following code:

In [ ]:
# print the number of nodes and edges in the network
print('Number of nodes', g.num_vertices())
print('Number of edges', g.num_edges())
print('Clustering coefficient', gt.local_clustering(g).fa.mean())
# Note that calculating the average path length exactly can be computationally
# expensive for large networks, so here we use the pseudo diameter as an
# approximation
print('Average path length', gt.pseudo_diameter(g)[0])

**Now try generating a ring lattice with 100 nodes and degree of 8 and calculate the average path length and clustering coefficient**

### Random (Erdos-Renyi) graphs
Real-world networks tend to have more random connections than regular lattices. One of the simplest random graph models is the Erdos-Renyi model.

In the Erdos-Renyi model every pair of nodes has the same probablity of having a link between them.

In [ ]:
g = gt.random_graph(100, lambda: np.random.poisson(10), directed=False)
gt.graph_draw(g)

**Compute and display the clustering coefficient, path length and degree distribution for this random graph. What do you notice?**

In [ ]:
print('Number of nodes', # insert code here)
print('Number of edges', # insert code here)
print('Clustering coefficient', # insert code here)
print('Average path length', # insert code here)

# plot a histogram of the degrees
degree_sequence = g.get_total_degrees(list(g.vertices()))
_ = plt.hist(degree_sequence, bins=10)
_ = plt.axvline(np.mean(degree_sequence), color='black')
_ = plt.title('Mean Degree = {:.2f}'.format(np.mean(degree_sequence)))

**Generate another random graph with 1000 nodes with the same expected degree and plot the degree distribution**

We use the binomial distribution to determine the probability of observing a node with a degree of 15 or more.

In [ ]:
# number of nodes
n = 1000
# mean degree
c = 10
# probability of a connection
p = c/(n-1)
# probability of a node having at least 15 connections
test = binomtest(15, n-1, p, alternative='greater')
test.pvalue

**How many nodes are there in the network at least 15 connections?**

**How many nodes would you expect to have a degree of at least 15?**

The code below loads a network in face-to-face interactions between high school students during the first hour of the school day.

In [ ]:
!wget https://networks.skewed.de/net/sp_high_school/files/proximity.gt.zst
g_proximity = gt.load_graph("proximity.gt.zst", )

# create a graph view of the interactions in the first hour of the dataset
g_prox_hour1 = gt.GraphView(g_proximity, efilt = lambda e: g_proximity.ep.time[e] <= 1385982020 + 3600)


**Considering the degree distribution, can you perform a statistical test to show that the interaction patterns are not modelled well with a ER random graph?**

**Can you perform the same sort of tests using the clustering coefficient?**

### PART II: Small worlds and heavy tails

In the first part of this practical, we will consider some simple network models and the properties of the networks they generate.

The graph-tool documentation for these network models is [here](https://graph-tool.skewed.de/static/doc/generation.html). **It will be useful to refer to the documentation to understand and use the code below**

Recall from the lecture that there are a number of properties that real-world networks often exhibit:

1.   High clustering
2.   Short path lengths
3.   Heavy-tailed degree distributions

We will see that the following network models capture these properties to varying extents.

### Small-world networks

Small-world networks are networks that share properties of regular networks and random networks. Small-world networks are created by randomising a proportion of the links in a regular network, while preserving part of the regular structure.

![image.png](https://drive.google.com/uc?id=1P8ewNjJuGlnIvZp1RKF2WCQwAFaZ8AL5)



In [ ]:
n_networks = 14
rewire_probs = np.logspace(-4, 1, n_networks)
n_nodes = 1000
n_edges = 5000
g = gt.circular_graph(n_nodes, n_edges/n_nodes)

C0 = gt.global_clustering(g)[0]
L0 = gt.pseudo_diameter(g)[0]

clusts = np.empty(n_networks)
lengths = np.empty(n_networks)

for i in range(n_networks):
  # generate ring lattice
  g = gt.circular_graph(n_nodes, n_edges/n_nodes)
  p = rewire_probs[i]
  generation.random_rewire(g, n_iter=np.round(p*n_edges), edge_sweep=False)
  clusts[i] = gt.global_clustering(g)[0]/C0
  lengths[i] = gt.pseudo_diameter(g)[0]/L0

plt.semilogx(rewire_probs, clusts, 'o', label='clustering')
plt.semilogx(rewire_probs, lengths, 'o', label='diameter')
plt.legend()
_ = plt.xlabel('Rewire probability (p)')


**What range of $p$ produces "realistic" networks?**

### The Price Model (preferential attachment)

The models so far capture some important properties commonly observed in real-world networks, however they all create networks with a relatively homogenous degree distribution.

The Price model (or Barabási-Albert model, if undirected) creates networks based on a mechanism known as preferential attachment.

In [ ]:
g = gt.price_network(5000)

gt.graph_draw(g, pos=gt.sfdp_layout(g, cooling_step=0.99),
              vertex_fill_color=g.vertex_index, vertex_size=2,
              vcmap=mpl.cm.plasma,
              edge_pen_width=1)

Above the colours indicate the order in which the nodes were added to the network.

We can now plot the degree distribution on a log-log axes...

In [ ]:
# Let's plot the degree distribution
in_hist = gt.vertex_hist(g, "total")

y = in_hist[0]
err = np.sqrt(in_hist[0])
err[err >= y] = y[err >= y] - 1e-2

plt.figure(figsize=(6,4))

plt.errorbar(in_hist[1][:-1], in_hist[0], fmt="o", yerr=0, label="in")

plt.gca().set_yscale("log")
plt.gca().set_xscale("log")
plt.gca().set_ylim(1e-1, 1e5)
plt.gca().set_xlim(0.8, 1e3)
plt.subplots_adjust(left=0.2, bottom=0.2)
plt.xlabel("$k_{in}$")
plt.ylabel("$NP(k_{in})$")
plt.tight_layout()

### The configuration model

The configuration model is an extention of the random graph model in which an arbitrary degree sequence can be specified.

We can generate a configuration model network using the `random_graph` function in graph-tool, by specifying a function that generates a heavy-tailed distribution to set the node degrees.

However, an alternative is to use a network that already has a heavy-tailed degree sequence and rewire the edges such that the degrees are preserved.

In [ ]:
generation.random_rewire(g, model='configuration')
gt.graph_draw(g)

### The Friendship paradox

**Test the proxmity network from earlier to determine if the friendship paradox holds.**

In [ ]:
# load the data and remove parallel edges
g_proximity = gt.load_graph("proximity.gt.zst", )
gt.remove_parallel_edges(g_proximity)

# create an adjacency matrix
A = gt.adjacency(g_proximity)
# calculate a vector of degrees
k = np.sum(A, axis=1)
# calculate the mean of friends degrees
friend_degrees = # insert code here
# compare with the actual degrees
popular_nodes =  # insert code here
print('The proportion of nodes that have more friends than their average friend is', popular_nodes)

### PART III: The BESTest

The following gives an example on how to run the BESTest on the Lazega lawyers networks. Try out the code for yourself.

In [ ]:
# function to get entropy of a random partition
def random_entropy(g, n_groups):

  n = g.num_vertices()
  b = np.random.randint(n_groups, size=n)
  vprop_b = g.new_vertex_property("int")
  for i in range(n):
    v = g.vertex(i)
    vprop_b[v] = b[i]

  state = gt.BlockState(g, b=vprop_b)
  return state.entropy()


In [ ]:
!wget 'https://github.com/jgarciab/NetworkScience/raw/main/Data/law_firm.gt.zst'
g = gt.load_graph('law_firm.gt.zst')

In [ ]:
# create a graph with layer 1
u = gt.GraphView(g, efilt=lambda e: g.ep.layer[e] == 1)
g1 = gt.Graph(u, prune=True)
print(list(g1.vp.keys()))

In [ ]:
attr_name = 'nodeOffice'
state = gt.BlockState(g1, b=g1.vp[attr_name])
test_value = state.entropy()
print('observed test statistic (entropy)', test_value)

In [ ]:
n_samples = 1000
n_groups = len(set(g1.vp[attr_name]))
null_entropy = np.empty(n_samples)
for i in range(n_samples):
  null_entropy[i] = random_entropy(g1, n_groups)

In [ ]:
plt.hist(null_entropy, bins=50)
_ = plt.axvline(test_value, color='black')

**Can you also run the BESTest on the proximity network?**

### Part IV: Reconstructing networks from noisy data

In [ ]:
# load the data and remove parallel edges
g_proximity = gt.load_graph("proximity.gt.zst", )
gt.remove_parallel_edges(g_proximity)



**Try rewiring the edges to introduce some "noise" into the network structure. What happens to the clustering coefficient, path length and assortativity?**

In [ ]:
## Complete the code to calculate the clustering coefficient, average path length, and assortativity of the network as noise is added.

# make a copy of the graph to add noise
g_proximity_with_noise = g_proximity.copy()

# calculate the clustering coefficient
clustering = gt.local_clustering(g_proximity_with_noise).fa.mean()
# calculate the average path length
diameter = gt.pseudo_diameter(g_proximity_with_noise)[0]
# calculate the assortativity
assortativity = correlations.assortativity(g_proximity_with_noise, 'total')[0]

# add noise to the graph by rewiring the edges
generation.random_rewire(g_proximity_with_noise, n_iter=100, edge_sweep=False)

Now we will attempt to reconstruct the network from a noisy version.

First we generate a noisy version by rewiring 100 edges...

In [ ]:
# make a copy of the graph to add noise
g_proximity_with_noise = g_proximity.copy()
generation.random_rewire(g_proximity_with_noise, n_iter=100, edge_sweep=False)

# calculate the similarity between the original and the noisy graph
similarity = topology.similarity(g_proximity, g_proximity_with_noise)
print('Similarity', similarity)

# calculate clustering coefficient of original and noisy graph
clustering = gt.local_clustering(g_proximity).fa.mean()
clustering_with_noise = gt.local_clustering(g_proximity_with_noise).fa.mean()
print('Clustering coefficient', clustering)
print('Clustering coefficient with noise', clustering_with_noise)


Now we attempt the reconstruction:

In [ ]:
n = g_proximity_with_noise.new_ep("int", 1)   # number of measurements
x = g_proximity_with_noise.new_ep("int", 1)   # number of observations

# We inititialize MeasuredBlockState, assuming that each non-edge has
# been measured only once (n_default=1) and not observed (x_default=0)

state = gt.MeasuredBlockState(g_proximity_with_noise, n=n, n_default=1, x=x, x_default=0)

# We will first equilibrate the Markov chain
gt.mcmc_equilibrate(state, wait=100, mcmc_args=dict(niter=10))

# Now we collect the marginals for exactly 100,000 sweeps, at
# intervals of 10 sweeps:

u = None              # marginal posterior edge probabilities
bs = []               # partitions
cs = []               # average local clustering coefficient

def collect_marginals(s):
   global u, bs, cs
   u = s.collect_marginal(u)
   bstate = s.get_block_state()
   bs.append(bstate.levels[0].b.a.copy())
   cs.append(gt.local_clustering(s.get_graph()).fa.mean())

gt.mcmc_equilibrate(state, force_niter=10000, mcmc_args=dict(niter=10),
                    callback=collect_marginals)

eprob = u.ep.eprob
print("Estimated average local clustering: %g ± %g" % (np.mean(cs), np.std(cs)))

g_inferred = gt.GraphView(u, efilt = lambda e: u.ep.eprob[e] >= 0.5)
similarity = topology.similarity(g_proximity, g_inferred)
print('Similarity', similarity)


We can also attempt to do the reconstruction using multiple noisy measurements...

In [ ]:
# create another graph with noise
g_proximity_with_noise1 = g_proximity.copy()
generation.random_rewire(g_proximity_with_noise1, n_iter=1000, edge_sweep=False)

x1 = g_proximity_with_noise1.new_ep("int", 1)

for edge in g_proximity_with_noise1.edges():
    try:
        x[(edge.source(), edge.target())] += 1
        n[(edge.source(), edge.target())] += 1

    except IndexError:
        g_proximity_with_noise.add_edge(edge.source(), edge.target())
        x[(edge.source(), edge.target())] = 1

n_obs += 1


Now we combine the measurements to perform the reconstruction...

In [ ]:
n = g_proximity_with_noise.new_ep("int", n_obs)   # number of measurements

state = gt.MeasuredBlockState(g_proximity_with_noise, n=n, n_default=n_obs, x=x, x_default=0)

# We will first equilibrate the Markov chain
gt.mcmc_equilibrate(state, wait=100, mcmc_args=dict(niter=10))

# Now we collect the marginals for exactly 100,000 sweeps, at
# intervals of 10 sweeps:

u = None              # marginal posterior edge probabilities
bs = []               # partitions
cs = []               # average local clustering coefficient

def collect_marginals(s):
   global u, bs, cs
   u = s.collect_marginal(u)
   bstate = s.get_block_state()
   bs.append(bstate.levels[0].b.a.copy())
   cs.append(gt.local_clustering(s.get_graph()).fa.mean())

gt.mcmc_equilibrate(state, force_niter=10000, mcmc_args=dict(niter=10),
                    callback=collect_marginals)

eprob = u.ep.eprob
print("Estimated average local clustering: %g ± %g" % (np.mean(cs), np.std(cs)))

g_inferred = gt.GraphView(u, efilt = lambda e: u.ep.eprob[e] >= 0.5)
similarity = topology.similarity(g_proximity, g_inferred)
print('Similarity', similarity)
